# Lucka 15

In [1]:
import pandas as pd

långfil = "input_data/15.txt"
with open(långfil, "r") as f:
    sensorer_rådata = f.read().splitlines()


In [2]:
sensorer_df = pd.DataFrame([tuple(map(int, sensor.replace('=', ',').replace(':', ',').split(',' )[1::2])) for sensor in sensorer_rådata], columns=["x_sensor", "y_sensor", "x_sändare", "y_sändare"])
sensorer_df["radie"] = sensorer_df.apply(lambda rad: abs(rad["x_sensor"]-rad["x_sändare"]) + abs(rad["y_sensor"]-rad["y_sändare"]), axis=1)
sensorer_df

,x_sensor,y_sensor,x_sändare,y_sändare,radie
0,391282,2038170,-532461,2166525,1052098
1,3042382,3783761,3113582,3814857,102296
2,3444090,757238,2930045,2000000,1756807
3,971638,288172,935006,638195,386655
4,2175844,1879176,2930045,2000000,875025
5,3063103,3820576,3113582,3814857,56198
6,2591294,3667337,2768198,3762135,271702
7,2579773,3989626,2768198,3762135,415916
8,2887876,2106773,2930045,2000000,148942
9,2808659,3280271,2768198,3762135,522325


## Del 1

In [3]:
y=2_000_000

sensorer_df["avstånd_kant"] = sensorer_df.apply(lambda rad: max(0, rad["radie"] - abs(rad["y_sensor"] - y)), axis=1)
sensorer_df["fria"] = sensorer_df.apply(lambda rad: list(range(rad["x_sensor"]-rad["avstånd_kant"], rad["x_sensor"]+rad["avstånd_kant"]+1)) if rad["avstånd_kant"] else [], axis=1)

alla_x = set()
sensorer_df["fria"].apply(lambda v: alla_x.update(v))

sensorer_df.apply(lambda rad: alla_x.discard(rad["x_sändare"] if rad["y_sändare"] == y else None), axis=1)

len(alla_x) 

4961647

## Del 2

In [4]:
koordinat_min, koordinat_max = 0, 4_000_000

linjer_df = pd.DataFrame([], columns=["a", "b", "n"])

for _, (x, y, r) in sensorer_df.iloc[:, [0, 1, 4]].iterrows():
    linjer = [(-(-1)**i, int(y + (-1)**i*x + 2*(round((i*2)%3/2+0.1)-0.5)*(r+1))) for i in range(4)]

    for a, b in linjer:
        radindex =((linjer_df["a"] == a) & (linjer_df["b"] == b))

        if radindex.any():
            linjer_df.loc[radindex, "n"] += 1   

        else:
            linjer_df.loc[linjer_df.shape[0]+1] = [a, b, 1]

stigande_linjer = linjer_df.query("n >= 2 and a == 1") 
avtagande_linjer = linjer_df.query("n >= 2 and a == -1") 

bingo = False
for _, (a_1, b_1, _) in stigande_linjer.iterrows():
    for _, (a_2, b_2, _) in avtagande_linjer.iterrows():
        x =  (b_2 - b_1)/(a_1 - a_2)

        if not x % 1:
            x = int(x)
            y = a_1 * x + b_1

            if  koordinat_min <= x <= koordinat_max and koordinat_min <= y <= koordinat_max: 

                for _, (x_sensor, y_sensor, r) in sensorer_df.iloc[:, [0, 1, 4]].iterrows():
                    if abs(x-x_sensor) + abs(y-y_sensor) <= r:
                        break
                else:
                    print(f"Bingo! {x=} {y=}")
                    bingo = True
                    break
    if bingo:
        break
                    
4000000*x+y


Bingo! x=3068581 y=3017867


12274327017867